In [17]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
import logging
import re 
import requests  # Import requests module

# Logging configuration
logging.basicConfig(filename='scraping.log', level=logging.INFO)

# Create an empty DataFrame to hold all the data
combined_df = pd.DataFrame()

# Define the URL where the XLSX files are located
url = "https://www.nrb.org.np/category/monthly-statistics/?department=bfr"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links on the page
    links = soup.find_all('a')

    # Iterate through the links
    for link in links:
        href = link.get('href')

        if href and href.endswith('.xlsx'):
            absolute_url = urllib.parse.urljoin(url, href)
          
            df = pd.read_excel(absolute_url, sheet_name='C4', header=0)
            df = df.iloc[:, 2:] 
            df = df.iloc[1:] 
            df.reset_index(drop=True, inplace=True)
            
            df.columns = df.iloc[0]
            df = df.iloc[1:]
            df.reset_index(drop=True, inplace=True)
            df = df.transpose()
            df.columns = df.iloc[0]
            df = df.iloc[1:]
            df.reset_index(drop=True, inplace=True)
            df['source_link'] = href
            df['extracted_source_link'] = df['source_link'].str.extract(r'([^/]+)_Publish', flags=re.IGNORECASE)

            combined_df = pd.concat([combined_df, df], ignore_index=True)
            combined_df = combined_df.dropna(how='all')


combined_df.to_excel('MajorFinancialIndicator.xlsx', index=False)
